In [312]:
import os
import sys
from os.path import join
import json

from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import nfl_data_py as nfl

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, os.path.join(ROOT_DIR,'py'))

import util
from plot.plot_simple import plot_play_with_speed

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

with open("paths.json", 'r') as f:
    paths = json.load(f)

PROCESSED_DATA_PATH = paths['processed_data']

In [313]:
WEEK = 1

df_tracking = pd.read_pickle(join(PROCESSED_DATA_PATH, f'wk{WEEK}', 'tracking_final.pkl'))
df_game = pd.read_pickle(join(PROCESSED_DATA_PATH, f'wk{WEEK}', 'games_final.pkl'))
df_play = pd.read_pickle(join(PROCESSED_DATA_PATH, f'wk{WEEK}', 'play_final.pkl'))
df_player_play = pd.read_pickle(join(PROCESSED_DATA_PATH, f'wk{WEEK}', 'player_play_final.pkl'))
df_player = pd.read_pickle(join(PROCESSED_DATA_PATH, 'players.pkl'))
df_team = pd.read_pickle(join(PROCESSED_DATA_PATH, 'teams.pkl'))

In [314]:
# drop everything before first line_set event
cols = [
    'game_play_id', 'frame_id','frame_type', 'event_new', 'nfl_id', 'position_by_loc', 
    'extra_oline_box_left', 'oline_box_left', 'center_x_at_line_set', 'oline_box_right', 
    'extra_oline_box_right', 'x', 'y', 's', 'a', 'o', 'dir', 'motion_player'
]
df_motion_and_shifts = df_tracking.query(
    'frame_id >= first_line_set_fid and ' +
    'frame_id <= ball_snap_fid + 10 and ' +
    'offense'
)[cols].copy()

# line_set window is each set of frame between line_set events
df_motion_and_shifts = df_motion_and_shifts.sort_values(['game_play_id','frame_id'])
line_set_windows = (
    df_motion_and_shifts
    .query('event_new == "line_set"')  # Only consider 'line_set' events
    .drop_duplicates(['game_play_id', 'frame_id'])  # Ensure unique frame per event
    .assign(line_set_window_number=lambda x: x.groupby('game_play_id').cumcount())  # Number line_set events within each game_play_id
)

# Step 3: Merge back to the original DataFrame
df_motion_and_shifts = df_motion_and_shifts.merge(
    line_set_windows[['game_play_id', 'frame_id', 'line_set_window_number']],
    on=['game_play_id', 'frame_id'],
    how='left'  # Merge without dropping rows from the original DataFrame
)

# drop plays without at occurence of motion_player
motion_gids = (
    df_motion_and_shifts
    .query('motion_player')
    .game_play_id
    .unique()
    .tolist()
)
df_motion_and_shifts = df_motion_and_shifts[df_motion_and_shifts.game_play_id.isin(motion_gids)]

df_motion_and_shifts['line_set_window_number'] = df_motion_and_shifts['line_set_window_number'].fillna(method='ffill').astype(int)

qb_x_last_line_set = (
    df_tracking
    .query('frame_id==last_line_set_fid and position=="QB"')
    .set_index('game_play_id')
    [['x','y']]
    .rename(columns={'x':'qb_x_last_line_set', 'y':'qb_y_last_line_set'})
    .reset_index()
)

df_motion_and_shifts = df_motion_and_shifts.merge(qb_x_last_line_set, on='game_play_id')

In [315]:
MOVING_THRESHOLD = 1.0
N_FRAMES_NOT_MOVING = 15

tqdm.pandas()

df_motion_cpy = df_motion_and_shifts.query('motion_player').copy()

# Step 2: Define the 'moving' column based on the threshold
df_motion_cpy['moving'] = df_motion_cpy['s'] >= MOVING_THRESHOLD

# Step 3: Create motion_frame column
def find_motion_frames(group):
    motion_frame = [False] * len(group)
    moving_array = group['moving'].values
    
    # Work backwards from the last frame
    last_idx = group.reset_index(drop=True).query('frame_type=="SNAP"').index[0] - 1

    # Find the first "moving" frame starting from the end
    first_moving_idx = None
    for idx in range(last_idx, -1, -1):
        if moving_array[idx]:
            first_moving_idx = idx
            break

    last_moving_idx = None
    if first_moving_idx is not None:
        for idx in range(first_moving_idx, last_idx + 1):
            if moving_array[idx]:
                last_moving_idx = idx
    
    if first_moving_idx is not None:
        # Set all frames after the first moving frame to True
        for idx in range(first_moving_idx, last_idx + 1):
            motion_frame[idx] = True

        # Move backwards, stopping when 10 consecutive frames are not moving
        consecutive_not_moving = 0
        for idx in range(first_moving_idx, -1, -1):
            if not moving_array[idx]:
                consecutive_not_moving += 1
                if consecutive_not_moving > N_FRAMES_NOT_MOVING:
                    # Stop marking frames as part of the motion
                    break
                continue
            else:
                for i in range(1, consecutive_not_moving+1):
                    motion_frame[idx+i] = True
                consecutive_not_moving = 0  # Reset counter if moving
            motion_frame[idx] = True

    # Set all frames to False from last_moving_idx to the end
    if last_moving_idx is not None:
        for idx in range(last_moving_idx + 1, len(group)):
            motion_frame[idx] = False

    # Update the group's motion_frame column
    group['motion_frame'] = motion_frame
    return group

# Apply the logic group-wise for each game_play_id
df_motion_cpy = df_motion_cpy.groupby('game_play_id', group_keys=False).progress_apply(find_motion_frames)

# drop game_play_ids with no motion frames
motion_gids = df_motion_cpy.query('motion_frame').game_play_id.unique()
df_motion_cpy = df_motion_cpy.query('game_play_id in @motion_gids')

100%|██████████| 266/266 [00:00<00:00, 993.18it/s] 


In [316]:
if 'motion_frame' in df_tracking.columns:
    df_tracking.drop(columns='motion_frame', inplace=True)

df_tracking = df_tracking.merge(df_motion_cpy[['game_play_id','frame_id','motion_frame']], on=['game_play_id','frame_id'], how='left')
df_tracking['motion_frame'] = df_tracking['motion_frame'].fillna(False)

In [317]:
cols = ['game_play_id', 'nfl_id', 'frame_id', 'frame_type', 'event_new', 'motion_frame',
    'position_by_loc', 'extra_oline_box_left', 'oline_box_left', 'center_x_at_line_set', 
    'oline_box_right', 'extra_oline_box_right', 'qb_x_last_line_set','qb_y_last_line_set', 
    'x', 'y', 's', 'a', 'o', 'dir', 'line_set_window_number','moving']
df_motion = df_motion_cpy.query('motion_player')[cols]

df_motion = (
    df_motion.merge(
        df_player_play[['game_play_id','nfl_id','had_rush_attempt']],
        how='left',
        on=['game_play_id','nfl_id']
    )
)

if 'absolute_yardline_number' not in df_motion.columns:
    df_motion = df_motion.merge(
        df_play[['game_play_id','absolute_yardline_number']],
        on='game_play_id',
        how='left'
    )

df_motion_first_frame = df_motion.drop_duplicates('game_play_id', keep='first').copy()

# Classify First Frame as "Off-Line Y"
df_motion_first_frame['initial_alignment'] = np.where(
    df_motion_first_frame['position_by_loc'].isin(['TE','RB','FB','WR']) &
    (
        (df_motion_first_frame['y'] < df_motion_first_frame['absolute_yardline_number']-1) &
        (df_motion_first_frame['y'] > df_motion_first_frame['absolute_yardline_number'] - 4)
    ),
    np.where(
        (
            (df_motion_first_frame['x'] < df_motion_first_frame['extra_oline_box_left']) &
            (df_motion_first_frame['x'] > df_motion_first_frame['extra_oline_box_left'] - 3)
        ) |
        (
            (df_motion_first_frame['x'] > df_motion_first_frame['extra_oline_box_right']) &
            (df_motion_first_frame['x'] < df_motion_first_frame['extra_oline_box_right'] + 3)
        ),
        'Off-Line Y',
        None
    ),
    None
)

if 'initial_alignment' in df_motion.columns:
    df_motion.drop(columns='initial_alignment', inplace=True)
df_motion = df_motion.merge(
    df_motion_first_frame[['game_play_id','initial_alignment']], 
    on=['game_play_id'],
    how='left'
)
del df_motion_first_frame

# left side negative, right side positive
df_motion['dx_outside_oline'] = np.where(
    df_motion['x'] < df_motion['extra_oline_box_left'],
    df_motion['x'] - df_motion['extra_oline_box_left'],
    np.where(
        df_motion['x'] > df_motion['extra_oline_box_right'],
        df_motion['x'] - df_motion['extra_oline_box_right'],
        None
    )
)

# negative if closer to left sideline, positive if closer to right sideline
df_motion['dx_inside_oline'] = np.where(
    (df_motion['extra_oline_box_left'] < df_motion['x']) &
    (df_motion['x'] < df_motion['extra_oline_box_right']),
    np.where(
        df_motion['x'] < df_motion['center_x_at_line_set'],
        df_motion['extra_oline_box_left'] - df_motion['x'],
        df_motion['extra_oline_box_right'] - df_motion['x']
    ),
    None
)

# cross the line of scrimmage if play with motion_frame has both dx_outside_oline < 0 and dx_outside_oline > 0
df_motion['oline_side'] = np.where(
    df_motion['dx_outside_oline'] < 0,
    'outside-left',
    np.where(
        df_motion['dx_outside_oline'] > 0,
        'outside-right',
        np.where(
            (df_motion['dx_inside_oline'] < 0) &
            (df_motion['dx_inside_oline'] > -1),
            'inside-oline-left',
            np.where(
                (df_motion['dx_inside_oline'] > 0) &
                (df_motion['dx_inside_oline'] < 1),
                'inside-oline-right',
                np.where(
                    df_motion['dx_inside_oline'] >= 1,
                    'within-oline-right',
                    np.where(
                        df_motion['dx_inside_oline'] <= -1,
                        'within-oline-left',
                        None
                    )
                )
            )
        )
    )
)
outside_left_gids = (
    df_motion
    .query('motion_frame and oline_side=="outside-left"')
    .game_play_id
    .unique()
    .tolist()
)
outside_right_gids = (
    df_motion
    .query('motion_frame and oline_side=="outside-right"')
    .game_play_id
    .unique()
    .tolist()
)
crossing_oline_gids = (
    df_motion
    .query('game_play_id.isin(@outside_left_gids) and oline_side.isin(["inside-oline-right","outside-right"])')
    .game_play_id
    .unique()
    .tolist()
)
crossing_oline_gids += (
    df_motion
    .query('game_play_id.isin(@outside_right_gids) and oline_side.isin(["inside-oline-left","outside-left"])')
    .game_play_id
    .unique()
    .tolist()
)
df_motion['motion_crosses_over_oline'] = np.where(
    df_motion['game_play_id'].isin(crossing_oline_gids),
    True,
    False
)
del crossing_oline_gids, outside_left_gids, outside_right_gids

first_motion_frame_side = (
    df_motion
    .query('motion_frame')
    .drop_duplicates('game_play_id', keep='first')
    [['game_play_id','oline_side']]
    .rename(columns={'oline_side':'oline_side_first'})
)
df_motion = df_motion.merge(first_motion_frame_side, on='game_play_id', how='left')
del first_motion_frame_side
last_presnap_frame_side = (
    df_motion
    .query('frame_type=="BEFORE_SNAP"')
    .drop_duplicates('game_play_id', keep='last')
    [['game_play_id','oline_side']]
    .rename(columns={'oline_side':'oline_side_last_presnap'})
)
df_motion = df_motion.merge(last_presnap_frame_side, on='game_play_id', how='left')
del last_presnap_frame_side
motion_enters_oline_gids = (
    df_motion
    .query('motion_frame and oline_side.isin(["inside-oline-left","inside-oline-right","within-oline-left","within-oline-right"])')
    .game_play_id
    .unique()
    .tolist()
)
df_motion['motion_enters_oline'] = np.where(
    df_motion['game_play_id'].isin(motion_enters_oline_gids),
    True,
    False
)

# df_motion['moving_directly_forward'] = (df_motion['dir'] > 45) & (df_motion['dir'] <= 135)
# df_motion['moving_directly_left'] = (df_motion['dir'] > 135) & (df_motion['dir'] <= 225)
df_motion['moving_leftish'] = (df_motion['dir'] > 90) & (df_motion['dir'] <= 270)
# df_motion['moving_directly_backward'] = (df_motion['dir'] > 225) & (df_motion['dir'] <= 315)
# df_motion['moving_directly_right'] = (df_motion['dir'] > 315) | (df_motion['dir'] <= 45)
df_motion['moving_rightish'] = (df_motion['dir'] > 270) | (df_motion['dir'] <= 90)

df_motion['moving_left'] = np.where(
    df_motion['moving_leftish'],
    1,
    0
)
# Count direction changes with the three frame streak in that direction
filtered_motion = (
    df_motion
    .query('motion_frame') 
    .sort_values(['game_play_id', 'frame_id'])  # Ensure proper ordering
)
def count_direction_changes_and_group(group):
    directions = group['moving_left'].values
    rolling_streaks = pd.Series(directions).rolling(3).apply(lambda x: len(set(x)) == 1).fillna(0).astype(bool)
    
    group_id = []
    current_group = 0
    prev_direction = directions[0]
    
    for i in range(len(rolling_streaks)):
        if rolling_streaks[i]:  # Valid streak found
            current_direction = directions[i]
            if current_direction != prev_direction:  # Change in direction
                current_group += 1
                prev_direction = current_direction
        group_id.append(current_group)
    
    group = group.copy()
    group['direction_group_id'] = group_id
    n_changes = current_group  # Total changes are equal to the last group ID
    group['n_direction_changes'] = n_changes
    return group
direction_grouped = (
    filtered_motion
    .groupby('game_play_id', group_keys=False)  # Group by game_play_id
    .apply(count_direction_changes_and_group)
)
df_motion = df_motion.merge(
    direction_grouped[['game_play_id', 'frame_id', 'direction_group_id', 'n_direction_changes']],
    on=['game_play_id', 'frame_id'],
    how='left'
)

# group by game_play_id, then set all frmaes to n_direciton_chnages value
n_direction_changes = (
    df_motion
    .query('motion_frame')
    .groupby('game_play_id')
    .agg(
        n_direction_changes=('n_direction_changes','first')
    )
)
df_motion = (
    df_motion
    .drop(columns='n_direction_changes')
    .merge(n_direction_changes, on='game_play_id', how='left')
)

motion_dir = (
    df_motion
    .query('frame_type == "BEFORE_SNAP" and motion_frame')
    [['game_play_id','moving_leftish','moving_rightish']]
    .copy()
)

motion_dir = (
    motion_dir
    .groupby('game_play_id')
    .agg(
        # motion_forward_only=('moving_directly_forward','all'),
        # motion_left_only=('moving_directly_left','all'),
        # motion_backward_only=('moving_directly_backward','all'),
        # motion_right_only=('moving_directly_right','all'),
        motion_leftish_all=('moving_leftish','all'),
        motion_rightish_all=('moving_rightish','all')
    )
)

df_motion = df_motion.merge(motion_dir, on='game_play_id', how='left')

df_motion['same_motion_dir'] = np.select(
    [
        df_motion['motion_leftish_all'],
        df_motion['motion_rightish_all']
    ],
    [
        'left-all',
        'right-all'
    ],
    default=None
)

df_motion.drop(columns=['motion_leftish_all','motion_rightish_all'], inplace=True)

motion_dir_by_group = (
    df_motion
    .query('motion_frame')
    .groupby(['game_play_id','direction_group_id'])
    .agg(
        n_frame_leftish=('moving_leftish','sum'),
        n_frame_rightish=('moving_rightish','sum'),
        contains_motion_frame=('motion_frame','any')
    )
    .reset_index()
)
motion_dir_by_group['dir_smoothed'] = np.select(
    [
        motion_dir_by_group['n_frame_leftish'] >= motion_dir_by_group['n_frame_rightish'],
        motion_dir_by_group['n_frame_leftish'] < motion_dir_by_group['n_frame_rightish']
    ],
    [
        'left',
        'right'
    ],
    default=None
)

# motion dir on first line set window
motion_dir_by_group_first = (
    motion_dir_by_group
    .drop_duplicates('game_play_id', keep='first')
    .copy()
    .rename(columns={'dir_smoothed':'motion_dir_first'})
)
df_motion = df_motion.merge(
    motion_dir_by_group_first[['game_play_id','motion_dir_first']], 
    on='game_play_id', 
    how='left'
)

# motion dir on last line set window
motion_dir_by_group_last = (
    motion_dir_by_group
    .drop_duplicates('game_play_id', keep='last')
    .copy()
    .rename(columns={'dir_smoothed':'motion_dir_last'})
)
df_motion = df_motion.merge(
    motion_dir_by_group_last[['game_play_id','motion_dir_last']],
    on='game_play_id',
    how='left'
)


df_motion = df_motion.merge(
    motion_dir_by_group[['game_play_id','direction_group_id','dir_smoothed','contains_motion_frame']], 
    on=['game_play_id','direction_group_id'], 
    how='left'
)
df_motion['contains_motion_frame'] = df_motion['contains_motion_frame'].fillna(False)

#Add QB x,y to df_motion
df_motion = df_motion.merge(
    (
        df_tracking
        .query('position_by_loc=="QB"')
        [['game_play_id','frame_id','x','y']]
        .rename(columns={'x':'qb_x','y':'qb_y'})
    ),
    on=['game_play_id','frame_id'],
    how='left'
)

# find max dx on motion
x_min_max = (
    df_motion
    .query('motion_frame')
    .groupby('game_play_id')
    .agg(
        x_min=('x','min'),
        x_max=('x','max'),
    )
)
x_min_max['dx'] = x_min_max['x_max'] - x_min_max['x_min']
df_motion = df_motion.merge(x_min_max[['dx']], on='game_play_id', how='left')

# find max dy on motion
y_min_max = (
    df_motion
    .query('motion_frame')
    .groupby('game_play_id')
    .agg(
        y_min=('y','min'),
        y_max=('y','max'),
    )
)
y_min_max['dy'] = y_min_max['y_max'] - y_min_max['y_min']
df_motion = df_motion.merge(y_min_max[['dy']], on='game_play_id', how='left')

# behind center at snap
frame_at_snap = (
    df_motion[['game_play_id','nfl_id','center_x_at_line_set']].drop_duplicates()
    .merge(
        (
            df_tracking.query('frame_type=="SNAP"')
            [['game_play_id','nfl_id','x','y']]
            .rename(columns={'x':'x_at_line_set','y':'y_at_line_set'})
        ),
        on=['game_play_id','nfl_id'],
        how='left'
    )
)
frame_at_snap['behind_center_at_snap'] = np.where(
    (frame_at_snap['center_x_at_line_set'] - 0.5 < frame_at_snap['x_at_line_set']) &
    (frame_at_snap['x_at_line_set'] < frame_at_snap['center_x_at_line_set'] + 0.5),
    True,
    False
)
df_motion = df_motion.merge(frame_at_snap[['game_play_id','behind_center_at_snap']], on='game_play_id', how='left')

# Farthest back player at snap
farthest_back_at_snap = (
    df_tracking
    .query('frame_type=="SNAP"')
    .groupby('game_play_id')
    .agg(
        farthest_back_y=('y','min')
    ).reset_index()
)
motion_nfl_ids = df_motion.nfl_id.unique().tolist()
farthest_back = (
    frame_at_snap.query('nfl_id.isin(@motion_nfl_ids)').merge(farthest_back_at_snap, on='game_play_id', how='left')
)
farthest_back['farthest_back_at_snap'] = np.where(
    farthest_back['y_at_line_set'] == farthest_back['farthest_back_y'],
    True,
    False
)
df_motion = df_motion.merge(farthest_back[['game_play_id','farthest_back_at_snap']], on='game_play_id', how='left')
del farthest_back_at_snap, frame_at_snap

# set frame of first motion
first_motion_frame = (
    df_motion
    .query('motion_frame')
    .groupby('game_play_id')
    .agg(
        first_motion_fid=('frame_id','min')
    )
)
df_motion = df_motion.merge(first_motion_frame, on='game_play_id', how='left')

ball_snap_frame = (
    df_tracking
    .query('frame_type=="SNAP"')
    [['game_play_id','frame_id']]
    .rename(columns={'frame_id':'ball_snap_fid'})
    .drop_duplicates('game_play_id')
)
df_motion = df_motion.merge(ball_snap_frame, on='game_play_id', how='left')

df_motion.head()

,game_play_id,nfl_id,frame_id,frame_type,event_new,motion_frame,position_by_loc,extra_oline_box_left,oline_box_left,center_x_at_line_set,oline_box_right,extra_oline_box_right,qb_x_last_line_set,qb_y_last_line_set,x,y,s,a,o,dir,line_set_window_number,moving,had_rush_attempt,absolute_yardline_number,initial_alignment,dx_outside_oline,dx_inside_oline,oline_side,motion_crosses_over_oline,oline_side_first,oline_side_last_presnap,motion_enters_oline,moving_leftish,moving_rightish,moving_left,direction_group_id,n_direction_changes,same_motion_dir,motion_dir_first,motion_dir_last,dir_smoothed,contains_motion_frame,qb_x,qb_y,dx,dy,behind_center_at_snap,farthest_back_at_snap,first_motion_fid,ball_snap_fid
0,2022090800_101,53079.0,52,BEFORE_SNAP,line_set,False,FB,26.04,26.04,29.46,32.78,32.78,29.54,46.49,29.62,43.21,0.03,0.16,91.34,91.79,0,False,0,48,None,None,3.16,within-oline-right,True,outside-left,outside-left,False,True,False,1,NaN,0.0,right-all,right,right,NaN,False,29.67,46.41,1.32,0.13,False,False,100,106
1,2022090800_101,53079.0,53,BEFORE_SNAP,NaN,False,FB,26.04,26.04,29.46,32.78,32.78,29.54,46.49,29.60,43.21,0.07,0.57,95.49,160.58,0,False,0,48,None,None,3.18,within-oline-right,True,outside-left,outside-left,False,True,False,1,NaN,0.0,right-all,right,right,NaN,False,29.66,46.40,1.32,0.13,False,False,100,106
2,2022090800_101,53079.0,54,BEFORE_SNAP,NaN,False,FB,26.04,26.04,29.46,32.78,32.78,29.54,46.49,29.59,43.21,0.20,1.24,100.55,175.41,0,False,0,48,None,None,3.19,within-oline-right,True,outside-left,outside-left,False,True,False,1,NaN,0.0,right-all,right,right,NaN,False,29.65,46.39,1.32,0.13,False,False,100,106
3,2022090800_101,53079.0,55,BEFORE_SNAP,NaN,False,FB,26.04,26.04,29.46,32.78,32.78,29.54,46.49,29.55,43.22,0.41,1.64,105.78,178.76,0,False,0,48,None,None,3.23,within-oline-right,True,outside-left,outside-left,False,True,False,1,NaN,0.0,right-all,right,right,NaN,False,29.65,46.39,1.32,0.13,False,False,100,106
4,2022090800_101,53079.0,56,BEFORE_SNAP,NaN,False,FB,26.04,26.04,29.46,32.78,32.78,29.54,46.49,29.49,43.22,0.68,1.90,112.18,178.05,0,False,0,48,None,None,3.29,within-oline-right,True,outside-left,outside-left,False,True,False,1,NaN,0.0,right-all,right,right,NaN,False,29.67,46.39,1.32,0.13,False,False,100,106


In [318]:
# add feature of if motion is ahead or behind qb when palyer crosses qb path
motion_crossing_qb = (
    df_motion
    .query('frame_id > first_motion_fid and frame_id < ball_snap_fid + 10 and qb_x - 0.5 < x and x < qb_x + 0.5')
    [['game_play_id','direction_group_id','y','qb_y']]
    .drop_duplicates(['game_play_id','direction_group_id'], keep='first')
    .copy()
)
# fill na direction_group_id the previous value + 1
df_motion.loc[df_motion['direction_group_id'].isna(), 'direction_group_id'] = (
    df_motion['direction_group_id'].shift() + 1
)

motion_crossing_qb['motion_crossing_qb'] = np.where(
    motion_crossing_qb['y'] > motion_crossing_qb['qb_y'],
    'in-front-of-qb',
    'behind-qb'
)

if 'motion_crossing_qb' in df_motion.columns:
    df_motion.drop(columns='motion_crossing_qb', inplace=True)
df_motion = df_motion.merge(
    motion_crossing_qb[['game_play_id','direction_group_id','motion_crossing_qb']], 
    on=['game_play_id','direction_group_id'], 
    how='left'
)

df_motion.loc[df_motion['motion_crossing_qb'].isna() & df_motion['motion_frame'], 'motion_crossing_qb'] = 'not-crossing-qb'

last_motion_crossing_qb = (
    motion_crossing_qb
    .drop_duplicates('game_play_id', keep='last')
    .copy()
    .rename(columns={'motion_crossing_qb':'motion_crossing_qb_last'})
)

if 'motion_crossing_qb_last' in df_motion.columns:
    df_motion.drop(columns='motion_crossing_qb_last', inplace=True)
df_motion = df_motion.merge(
    last_motion_crossing_qb[['game_play_id','motion_crossing_qb_last']], 
    on='game_play_id', 
    how='left'
)

first_motion_crossing_qb = (
    motion_crossing_qb
    .drop_duplicates('game_play_id', keep='first')
    .copy()
    .rename(columns={'motion_crossing_qb':'motion_crossing_qb_first'})
)

if 'motion_crossing_qb_first' in df_motion.columns:
    df_motion.drop(columns='motion_crossing_qb_first', inplace=True)
df_motion = df_motion.merge(
    first_motion_crossing_qb[['game_play_id','motion_crossing_qb_first']], 
    on='game_play_id', 
    how='left'
)

# Classify Yo-Yo Motion and its variants

In [319]:
# Classify Yo-Yo motion (and one case of Orbit motion)

df_motion['motion_group'] = None
df_motion['motion_sub_group'] = None

# Yo-Yo motion: if changes direction more than once and starts motion moving towardes the qb
df_motion['motion_group'] = np.where(
    (
        (df_motion['n_direction_changes'] > 0) &
        (df_motion['dx'] > 2) &
        (
            (
                (df_motion['oline_side_first'] == "outside-left") & (df_motion['motion_dir_first'] == "right")
            ) |
            (
                (df_motion['oline_side_first'] == "outside-right") & (df_motion['motion_dir_first'] == "left")
            )
        )
    ),
    'Yo-Yo',
    df_motion['motion_group']
)

df_motion['motion_sub_group'] = np.where(
    (df_motion['motion_group'] == 'Yo-Yo'),
    np.where(
        (df_motion['n_direction_changes'] == 3),
        'Triple Yo-Yo',
        np.where(
            (df_motion['n_direction_changes'] == 2),
            'Double Yo-Yo',
            np.where(
                df_motion['behind_center_at_snap'] & ~df_motion['farthest_back_at_snap'],
                'Yo-Yo Lead',
                np.where(
                    (df_motion['motion_crossing_qb_first'] == 'in-front-of-qb') &
                    (df_motion['motion_crossing_qb_last'] == 'behind-qb'),
                    'Under Orbit',
                    np.where(
                        df_motion['motion_crosses_over_oline'],
                        'Full Yo-Yo',
                        np.where(
                            df_motion['motion_enters_oline'],
                            'Half Yo-Yo',
                            None
                        )
                    )
                )
            )
        )
    ),
    df_motion['motion_sub_group']
)

df_motion['motion_group'] = np.where(
    df_motion['motion_sub_group'] == 'Under Orbit',
    'Orbit',
    df_motion['motion_group']
)

# Classify Over Motion

In [320]:
df_motion['dx_dy_ratio'] = df_motion['dx'] / df_motion['dy']

if 'x_2_sec_after_snap' in df_motion.columns:
    df_motion.drop(columns=['x_2_sec_after_snap','y_2_sec_after_snap'], inplace=True)

df_motion = (
    df_motion
    .merge(
        (
            df_tracking.query('frame_id == ball_snap_fid + 20')
            [['game_play_id','nfl_id','x','y']]
            .rename(columns={'x':'x_2_sec_after_snap','y':'y_2_sec_after_snap'})
        ),
        on=['game_play_id','nfl_id'],
        how='left'
    )
)

# Feature for motion man has not reached center before snap (or not far past center)
x_loc_at_ball_snap = (
    df_motion
    .query('frame_id == ball_snap_fid')
    [['game_play_id','nfl_id','x']]
    .rename(columns={'x':'x_loc_at_ball_snap'})
)
df_motion = df_motion.merge(x_loc_at_ball_snap, on=['game_play_id','nfl_id'], how='left')
del x_loc_at_ball_snap

# This feature only applies to single direction motions
df_motion['ball_snap_before_passing_center'] = np.where(
    (
        (df_motion['oline_side_first'] == 'outside-left') &
        (df_motion['same_motion_dir'] == 'right-all') &
        (df_motion['x_loc_at_ball_snap'] < df_motion['center_x_at_line_set'] + 0.5)
    ) |
    (
        (df_motion['oline_side_first'] == 'outside-right') &
        (df_motion['same_motion_dir'] == 'left-all') &
        (df_motion['x_loc_at_ball_snap'] > df_motion['center_x_at_line_set'] - 0.5)
    ),
    True,
    False
)


df_motion['motion_group'] = np.where(
    (df_motion['n_direction_changes'] == 0) &
    (df_motion['y_2_sec_after_snap'] >= df_motion['absolute_yardline_number'] - 3) &
    (df_motion['dx_dy_ratio'] >= 2) &
    (df_motion['initial_alignment'] == 'Off-Line Y') &
    (
        (
            (df_motion['oline_side_first'] == 'outside-left') &
            (df_motion['motion_dir_first'] == 'right') &
            (df_motion['x_2_sec_after_snap'] >= df_motion['center_x_at_line_set'] + 1)
        ) |
        (
            (df_motion['oline_side_first'] == 'outside-right') &
            (df_motion['motion_dir_first'] == 'left') &
            (df_motion['x_2_sec_after_snap'] <= df_motion['center_x_at_line_set'] - 1)
        )
    ),
    'Over',
    df_motion['motion_group']
)

df_motion['motion_sub_group'] = np.where(
    (df_motion['motion_group'] == 'Over'),
    np.where(
        df_motion['ball_snap_before_passing_center'],
        'Ball Snap Before Passing Center',
        'Ball Snap After Passing Center'
    ),
    df_motion['motion_sub_group']
)

In [321]:
df_motion.head()

,game_play_id,nfl_id,frame_id,frame_type,event_new,motion_frame,position_by_loc,extra_oline_box_left,oline_box_left,center_x_at_line_set,oline_box_right,extra_oline_box_right,qb_x_last_line_set,qb_y_last_line_set,x,y,s,a,o,dir,line_set_window_number,moving,had_rush_attempt,absolute_yardline_number,initial_alignment,dx_outside_oline,dx_inside_oline,oline_side,motion_crosses_over_oline,oline_side_first,oline_side_last_presnap,motion_enters_oline,moving_leftish,moving_rightish,moving_left,direction_group_id,n_direction_changes,same_motion_dir,motion_dir_first,motion_dir_last,dir_smoothed,contains_motion_frame,qb_x,qb_y,dx,dy,behind_center_at_snap,farthest_back_at_snap,first_motion_fid,ball_snap_fid,motion_crossing_qb,motion_crossing_qb_last,motion_crossing_qb_first,motion_group,motion_sub_group,dx_dy_ratio,x_2_sec_after_snap,y_2_sec_after_snap,x_loc_at_ball_snap,ball_snap_before_passing_center
0,2022090800_101,53079.0,52,BEFORE_SNAP,line_set,False,FB,26.04,26.04,29.46,32.78,32.78,29.54,46.49,29.62,43.21,0.03,0.16,91.34,91.79,0,False,0,48,None,None,3.16,within-oline-right,True,outside-left,outside-left,False,True,False,1,NaN,0.0,right-all,right,right,NaN,False,29.67,46.41,1.32,0.13,False,False,100,106,behind-qb,behind-qb,behind-qb,None,None,10.153846,39.33,47.81,26.33,True
1,2022090800_101,53079.0,53,BEFORE_SNAP,NaN,False,FB,26.04,26.04,29.46,32.78,32.78,29.54,46.49,29.60,43.21,0.07,0.57,95.49,160.58,0,False,0,48,None,None,3.18,within-oline-right,True,outside-left,outside-left,False,True,False,1,NaN,0.0,right-all,right,right,NaN,False,29.66,46.40,1.32,0.13,False,False,100,106,behind-qb,behind-qb,behind-qb,None,None,10.153846,39.33,47.81,26.33,True
2,2022090800_101,53079.0,54,BEFORE_SNAP,NaN,False,FB,26.04,26.04,29.46,32.78,32.78,29.54,46.49,29.59,43.21,0.20,1.24,100.55,175.41,0,False,0,48,None,None,3.19,within-oline-right,True,outside-left,outside-left,False,True,False,1,NaN,0.0,right-all,right,right,NaN,False,29.65,46.39,1.32,0.13,False,False,100,106,behind-qb,behind-qb,behind-qb,None,None,10.153846,39.33,47.81,26.33,True
3,2022090800_101,53079.0,55,BEFORE_SNAP,NaN,False,FB,26.04,26.04,29.46,32.78,32.78,29.54,46.49,29.55,43.22,0.41,1.64,105.78,178.76,0,False,0,48,None,None,3.23,within-oline-right,True,outside-left,outside-left,False,True,False,1,NaN,0.0,right-all,right,right,NaN,False,29.65,46.39,1.32,0.13,False,False,100,106,behind-qb,behind-qb,behind-qb,None,None,10.153846,39.33,47.81,26.33,True
4,2022090800_101,53079.0,56,BEFORE_SNAP,NaN,False,FB,26.04,26.04,29.46,32.78,32.78,29.54,46.49,29.49,43.22,0.68,1.90,112.18,178.05,0,False,0,48,None,None,3.29,within-oline-right,True,outside-left,outside-left,False,True,False,1,NaN,0.0,right-all,right,right,NaN,False,29.67,46.39,1.32,0.13,False,False,100,106,behind-qb,behind-qb,behind-qb,None,None,10.153846,39.33,47.81,26.33,True


# Classify Orbit, Fly, & Jet Motion

In [322]:
cols = ['game_play_id', 'nfl_id', 'first_motion_fid', 'ball_snap_fid','position_by_loc','oline_box_left','oline_box_right',
        'oline_side_first','same_motion_dir','initial_alignment','n_direction_changes','dx','dy',
        'dx_dy_ratio', 'qb_x','qb_y','absolute_yardline_number',
        'motion_group','motion_sub_group']
df_motion_all_frames = (
    df_motion.query('motion_frame')[cols].drop_duplicates(['game_play_id'])
    .merge(
        df_tracking[['game_play_id','nfl_id','frame_id','event_new','frame_type','x','y','s','a','dir']],
        on=['game_play_id','nfl_id'],
        how='left'
    )
    .merge(
        (
            df_tracking[['game_play_id','frame_id','position_by_loc','y']]
            .query('position_by_loc=="QB"')
            .rename(columns={'y':'qb_y_at_frame'})
            .drop(columns='position_by_loc')
        ),
        on=['game_play_id','frame_id'],
        how='left'
    )
)

# column for if motion crosses to other side of oline
if 'max_x_within_3_sec_post_snap' in df_motion_all_frames.columns:
    df_motion_all_frames.drop(
        columns=['max_x_within_3_sec_post_snap','min_x_within_3_sec_post_snap'], 
        inplace=True
    )

max_x_within_3_sec_post_snap = (
    df_motion_all_frames
    .query('ball_snap_fid <= frame_id <= ball_snap_fid + 30 and y <= absolute_yardline_number')
    [['game_play_id','x']]
    .sort_values('x', ascending=False)
    .drop_duplicates('game_play_id', keep='first')
    .rename(columns={'x':'max_x_within_3_sec_post_snap'})
)
min_x_within_3_sec_post_snap = (
    df_motion_all_frames
    .query('ball_snap_fid <= frame_id <= ball_snap_fid + 30 and y <= absolute_yardline_number')
    [['game_play_id','x']]
    .sort_values('x', ascending=True)
    .drop_duplicates('game_play_id', keep='first')
    .rename(columns={'x':'min_x_within_3_sec_post_snap'})
)
df_motion_all_frames = df_motion_all_frames.merge(max_x_within_3_sec_post_snap, on='game_play_id', how='left')
df_motion_all_frames = df_motion_all_frames.merge(min_x_within_3_sec_post_snap, on='game_play_id', how='left')
del max_x_within_3_sec_post_snap, min_x_within_3_sec_post_snap

# Figure out if motion player starts next to or ahead of QB
motion_starts_next_to_qb = (
    df_motion_all_frames
    .query('first_motion_fid == frame_id')
    [['game_play_id','frame_id','y']]
    .merge(
        df_tracking.query('position_by_loc=="QB"')[['game_play_id','frame_id','y']].rename(columns={'y':'qb_y'}),
        on=['game_play_id', 'frame_id'],
        how='left'
    )
)
motion_starts_next_to_qb['motion_player_starts_next_to_or_ahead_of_qb'] = np.where(
    (motion_starts_next_to_qb['qb_y'] - motion_starts_next_to_qb['y'] <= 0.5),
    True,
    False
)
df_motion_all_frames = df_motion_all_frames.merge(
    motion_starts_next_to_qb[['game_play_id','motion_player_starts_next_to_or_ahead_of_qb']],
    on='game_play_id', 
    how='left'
)
del motion_starts_next_to_qb

if 'motion_crosses_to_other_side' in df_motion_all_frames.columns:
    df_motion_all_frames.drop(columns='motion_crosses_to_other_side', inplace=True)

df_motion_all_frames['motion_crosses_to_other_side'] = np.where(
    (df_motion_all_frames['n_direction_changes'] == 0) &
    (
        (
            (df_motion_all_frames['oline_side_first'].isin(['outside-left','inside-oline-left','within-oline-left'])) &
            (df_motion_all_frames['same_motion_dir'] == 'right-all') &
            (df_motion_all_frames['oline_box_right'] < df_motion_all_frames['max_x_within_3_sec_post_snap'])
        ) |
        (
            (df_motion_all_frames['oline_side_first'].isin(['outside-right','inside-oline-right','within-oline-right'])) &
            (df_motion_all_frames['same_motion_dir'] == 'left-all') &
            (df_motion_all_frames['min_x_within_3_sec_post_snap'] < df_motion_all_frames['oline_box_left'])
        )
    ),
    True,
    False
)

df_motion_crossing_oline = (
    df_motion_all_frames
    .query('motion_crosses_to_other_side')
    .copy()
)
del df_motion_all_frames

# find the frame crossing the qb (need to redo in front or behind because above limits it to 30 second after snap)
motion_crossing_qb = (
    df_motion_crossing_oline
    .query('first_motion_fid <= frame_id and frame_id <= ball_snap_fid + 30 and qb_x - 0.5 < x and x < qb_x + 0.5')
    [['game_play_id','frame_id','x','y','qb_x','qb_y_at_frame']]
    .drop_duplicates('game_play_id', keep='first')
    .copy()
)
motion_crossing_qb['motion_behind_qb'] = np.where(
    motion_crossing_qb['qb_y_at_frame'] > motion_crossing_qb['y'],
    True,
    False
)
motion_crossing_qb['dx'] = (motion_crossing_qb['x'] - motion_crossing_qb['qb_x']).abs()
motion_crossing_qb = (
    motion_crossing_qb
    .sort_values('dx')
    .drop_duplicates('game_play_id', keep='first')
    .rename(columns={'frame_id':'frame_id_crossing_qb'})
)

df_motion_crossing_oline = df_motion_crossing_oline.merge(
    motion_crossing_qb[['game_play_id','motion_behind_qb','frame_id_crossing_qb']], 
    on='game_play_id', 
    how='left'
)

df_arc_motion = (
    df_motion_crossing_oline
    .query('first_motion_fid <= frame_id and frame_id <= frame_id_crossing_qb')
    [['game_play_id','x','y']]
)

# Assuming df_arc_motion is already defined
def calculate_path_distance(group):
    ''' Total distance traveled along the path and Euclidean distance between 
    first and last points '''
    path_distance = np.sum(
        np.sqrt(np.diff(group['x'])**2 + np.diff(group['y'])**2)
    )
    # Compute straight-line distance between first and last points
    straight_line_distance = np.sqrt(
        (group['x'].iloc[-1] - group['x'].iloc[0])**2 +
        (group['y'].iloc[-1] - group['y'].iloc[0])**2
    )
    return pd.Series({
        'path_distance': path_distance,
        'straight_line_distance': straight_line_distance,
        'dy_motion_to_qb': group['y'].iloc[-1] - group['y'].iloc[0]
    })

# Apply the function to the DataFrame
result = (
    df_arc_motion
    .groupby('game_play_id')
    .apply(calculate_path_distance)
    .reset_index()
)

df_arc_motion = df_arc_motion.merge(result, on='game_play_id', how='left')
del result

df_arc_motion['path_straight_ratio'] = df_arc_motion['path_distance'] / df_arc_motion['straight_line_distance']

df_motion_crossing_oline = df_motion_crossing_oline.merge(
    df_arc_motion[['game_play_id','path_distance','path_straight_ratio','dy_motion_to_qb']], 
    on='game_play_id', 
    how='left'
)

df_motion_crossing_oline['motion_group'] = np.where(
    df_motion_crossing_oline.motion_group.isna(),
    np.where(
        df_motion_crossing_oline['motion_behind_qb'] &
        (
            (df_motion_crossing_oline['dy_motion_to_qb'] <= -2) |
            (df_motion_crossing_oline['path_straight_ratio'] >= 1.005) |
            (df_motion_crossing_oline['motion_player_starts_next_to_or_ahead_of_qb'])
        ),
        'Orbit',
        np.where(
            df_motion_crossing_oline['dx_dy_ratio'] >= 2,
            np.where(
                df_motion_crossing_oline['frame_id_crossing_qb'] <= df_motion_crossing_oline['ball_snap_fid'] - 5,
                'Fly',
                'Jet'
            ),
            np.nan
        )
    ),
    df_motion_crossing_oline.motion_group
)
df_motion_crossing_oline.loc[
    (~df_motion_crossing_oline.motion_group.isna() & df_motion.motion_sub_group.isna()), 
    'motion_sub_group'
] = "Standard"

df_motion = (
    df_motion
    .merge(
        (
            df_motion_crossing_oline
            [['game_play_id','motion_group','motion_sub_group']]
            .drop_duplicates('game_play_id')
        ),
        on='game_play_id',
        how='left',
        suffixes=('','_x')
    )
)

df_motion['motion_group'] = np.where(
    df_motion['motion_group'].isna(),
    df_motion['motion_group_x'],
    df_motion['motion_group']
)
df_motion['motion_sub_group'] = np.where(
    df_motion['motion_sub_group'].isna(),
    df_motion['motion_sub_group_x'],
    df_motion['motion_sub_group']
)
df_motion.drop(columns=['motion_group_x','motion_sub_group_x'], inplace=True)

# Classify Glide Motion

In [323]:
cols = ['game_play_id', 'nfl_id', 'first_motion_fid','oline_box_left','oline_box_right',
        'oline_side_first','same_motion_dir','dx_dy_ratio',
        'motion_group','motion_sub_group']
df_motion_all_frames = (
    df_motion.query('motion_frame')[cols].drop_duplicates(['game_play_id'])
    .merge(
        df_tracking[['game_play_id','nfl_id','frame_id','event_new','frame_type',
                     'absolute_yardline_number','x','y','s','a','dir']],
        on=['game_play_id','nfl_id'],
        how='left'
    )
)
x_mins = (
    df_motion_all_frames
    .query('y <= absolute_yardline_number - 1 and frame_id >= first_motion_fid')
    [['game_play_id','x']]
    .sort_values('x')
    .drop_duplicates('game_play_id', keep='first')
    .rename(columns={'x':'x_motion_min'})
)
x_maxs = (
    df_motion_all_frames
    .query('y <= absolute_yardline_number - 1 and frame_id >= first_motion_fid')
    [['game_play_id','x']]
    .sort_values('x')
    .drop_duplicates('game_play_id', keep='last')
    .rename(columns={'x':'x_motion_max'})
)
x_at_ball_snap = (
    df_motion_all_frames
    .query('frame_type=="SNAP"')
    [['game_play_id','x']]
    .rename(columns={'x':'x_at_ball_snap'})
)
df_motion_all_frames = df_motion_all_frames.merge(x_mins, on='game_play_id', how='left')
df_motion_all_frames = df_motion_all_frames.merge(x_maxs, on='game_play_id', how='left')
df_motion_all_frames = df_motion_all_frames.merge(x_at_ball_snap, on='game_play_id', how='left')
df_motion_all_frames['x_motion_max'] = df_motion_all_frames['x_motion_max'].fillna(df_motion_all_frames['x_at_ball_snap'])
del x_mins, x_maxs

In [324]:
qry = df_motion_all_frames.copy()
glide_gids = qry[(
    qry['motion_group'].isna() &
    (qry['dx_dy_ratio'] > 3) &
    (
        (
            (qry['oline_side_first'] == 'outside-left') &
            (qry['same_motion_dir'] == 'right-all') & 
            (qry['x_motion_max'] < qry['oline_box_left'] + 1)
        ) |
        (
            (qry['oline_side_first'] == 'outside-right') &
            (qry['same_motion_dir'] == 'left-all') &
            (qry['x_motion_min'] > qry['oline_box_right'] - 1)
        )
    )
)].game_play_id.unique()

df_motion.loc[df_motion['game_play_id'].isin(glide_gids), 'motion_group'] = 'Glide'
df_motion.loc[df_motion['game_play_id'].isin(glide_gids), 'motion_sub_group'] = 'Standard'

# Classify Off-Line-Y In Motion

In [325]:
drop_cols = ['ball_x','ball_y','euclidean_dist_to_ball','dx_to_ball','dy_to_ball','dist_to_ball',
            'oline_box_left','oline_box_right','first_line_set_fid',
            'y_min_oline_left','y_min_oline_right','extra_on_oline','extra_oline_box_left',
            'extra_oline_box_right','center_x_at_line_set']

for col in drop_cols:
    if col in df_tracking.columns:
        df_tracking.drop(columns=col, inplace=True)

In [326]:
# Feature of the min and max x values of the 5 offensive linemen at the snap
oline_at_ls = (
    df_tracking
    .query('frame_id == last_line_set_fid and position_by_loc.isin(["LT","LG","C","RG","RT"])')
    [['game_play_id','position_by_loc','x','y']]
)
oline_x_min = oline_at_ls.groupby('game_play_id').x.min().reset_index().rename(columns={'x':'oline_x_left_at_ls'})
oline_x_max = oline_at_ls.groupby('game_play_id').x.max().reset_index().rename(columns={'x':'oline_x_right_at_ls'})
oline_y_min_right = (
    oline_at_ls
    .query('position_by_loc.isin(["C","RG","RT"])')
    .groupby('game_play_id')
    .y.min()
    .reset_index()
    .rename(columns={'y':'oline_y_min_right_at_ls'})
)
oline_y_min_left = (
    oline_at_ls
    .query('position_by_loc.isin(["LT","LG","C"])')
    .groupby('game_play_id')
    .y.min()
    .reset_index()
    .rename(columns={'y':'oline_y_min_left_at_ls'})
)
if 'oline_x_left_at_ls' in df_tracking.columns:
    df_tracking.drop(columns=['oline_x_left_at_ls'], inplace=True)
if 'oline_x_right_at_ls' in df_tracking.columns:
    df_tracking.drop(columns=['oline_x_right_at_ls'], inplace=True)
if 'oline_y_min_left_at_ls' in df_tracking.columns:
    df_tracking.drop(columns=['oline_y_min_left_at_ls'], inplace=True)
if 'oline_y_min_right_at_ls' in df_tracking.columns:
    df_tracking.drop(columns=['oline_y_min_right_at_ls'], inplace=True)
df_tracking = (
    df_tracking
    .merge(oline_x_min, on='game_play_id', how='left')
    .merge(oline_x_max, on='game_play_id', how='left')
    .merge(oline_y_min_left, on='game_play_id', how='left')
    .merge(oline_y_min_right, on='game_play_id', how='left')
)
del oline_at_ls, oline_x_min, oline_x_max, oline_y_min_left, oline_y_min_right

In [327]:
# Feature of Center x at ball snap
center_at_ls = (
    df_tracking
    .query('frame_id == last_line_set_fid and position_by_loc == "C"')
    [['game_play_id','x']]
    .rename(columns={'x':'center_x_at_ls'})
)
if 'center_x_at_ls' in df_tracking.columns:
    df_tracking.drop(columns='center_x_at_ls', inplace=True)
df_tracking = df_tracking.merge(center_at_ls, on='game_play_id', how='left')
del center_at_ls

In [328]:
# Set last line set to first motion frame if it is motion play
df_tracking = df_tracking.merge(
    (
        df_motion[['game_play_id','first_motion_fid']]
        .drop_duplicates('game_play_id')
        .rename(columns={'first_motion_fid':'last_line_set_fid'})
    ),
    on='game_play_id',
    how='left',
    suffixes=('','_new')
)
df_tracking['last_line_set_fid'] = np.where(
    df_tracking['last_line_set_fid_new'].notna(),
    df_tracking['last_line_set_fid_new'],
    df_tracking['last_line_set_fid']
)

In [329]:
# Label Extra Players acting as part of the OLine

MAX_YARDS_BEHIND_OLINE = 2
MAX_YARDS_NEXT_TO_OLINE = 2.5

# Initialize New Columns
df_tracking['extra_on_oline_x_left_at_ls'] = df_tracking.oline_x_left_at_ls
df_tracking['extra_on_oline_x_right_at_ls'] = df_tracking.oline_x_right_at_ls
df_tracking['extra_on_oline_y_min_left_at_ls'] = df_tracking.oline_y_min_left_at_ls
df_tracking['extra_on_oline_y_min_right_at_ls'] = df_tracking.oline_y_min_right_at_ls
df_tracking['on_oline'] = np.where(
    df_tracking.position_by_loc.isin(["LT","LG","C","RG","RT"]),
    True,
    False
)

# Loop until no new players are added to the oline
extra_on_oline = pd.DataFrame()
extra_on_oline_len = 1
while extra_on_oline_len != len(extra_on_oline):
    extra_on_oline = (
        df_tracking
        .query('(position_by_loc != "QB") and ~on_oline and offense and frame_id == last_line_set_fid')
        [['game_play_id','nfl_id','x','y','s', 'center_x_at_ls',
        'extra_on_oline_y_min_left_at_ls','extra_on_oline_y_min_right_at_ls',
        'extra_on_oline_x_left_at_ls','extra_on_oline_x_right_at_ls','on_oline']]
    )

    # Save initial length to check if we need to run the loop again
    extra_on_oline_len = len(extra_on_oline)

    extra_on_oline['on_oline'] = np.where(
        (
            (extra_on_oline.x < extra_on_oline.extra_on_oline_x_left_at_ls) &
            (extra_on_oline.x >= extra_on_oline.extra_on_oline_x_left_at_ls - MAX_YARDS_NEXT_TO_OLINE) &
            (extra_on_oline.y >= extra_on_oline.extra_on_oline_y_min_left_at_ls - MAX_YARDS_BEHIND_OLINE)
        ) |
        (
            (extra_on_oline.x > extra_on_oline.extra_on_oline_x_right_at_ls) &
            (extra_on_oline.x <= extra_on_oline.extra_on_oline_x_right_at_ls + MAX_YARDS_NEXT_TO_OLINE) &
            (extra_on_oline.y >= extra_on_oline.extra_on_oline_y_min_right_at_ls - MAX_YARDS_BEHIND_OLINE)
        ),
        True,
        False
    )

    # Update oline box left for the new players on the oline left side
    extra_on_online_x_min = (
        extra_on_oline
        .query('on_oline and x < center_x_at_ls')
        [['game_play_id','nfl_id','x']]
        .groupby('game_play_id')
        .x.min()
        .reset_index()
        .rename(columns={'x':'extra_on_oline_x_left_at_ls'})
    )
    df_tracking = (
        df_tracking
        .merge(
            extra_on_online_x_min[['game_play_id','extra_on_oline_x_left_at_ls']], 
            on='game_play_id', 
            how='left', 
            suffixes=('','_new')
        )
        .assign(
            extra_on_oline_x_left_at_ls = lambda x: np.where(
                x.extra_on_oline_x_left_at_ls_new.notnull() &
                (x.extra_on_oline_x_left_at_ls_new < x.extra_on_oline_x_left_at_ls),
                x.extra_on_oline_x_left_at_ls_new,
                x.extra_on_oline_x_left_at_ls
            )
        )
        .drop(columns='extra_on_oline_x_left_at_ls_new')
    )

    # Update oline box right for the new players on the oline right side
    extra_on_online_x_max = (
        extra_on_oline
        .query('on_oline and x > center_x_at_ls')
        [['game_play_id','nfl_id','x']]
        .groupby('game_play_id')
        .x.max()
        .reset_index()
        .rename(columns={'x':'extra_on_oline_x_right_at_ls'})
    )
    df_tracking = (
        df_tracking
        .merge(
            extra_on_online_x_max[['game_play_id','extra_on_oline_x_right_at_ls']], 
            on='game_play_id', 
            how='left', 
            suffixes=('','_new')
        )
        .assign(
            extra_on_oline_x_right_at_ls = lambda x: np.where(
                x.extra_on_oline_x_right_at_ls_new.notnull() &
                (x.extra_on_oline_x_right_at_ls_new > x.extra_on_oline_x_right_at_ls),
                x.extra_on_oline_x_right_at_ls_new,
                x.extra_on_oline_x_right_at_ls
            )
        )
        .drop(columns='extra_on_oline_x_right_at_ls_new')
    )

    # Update on_oline for the new players on the oline
    on_oline = extra_on_oline.query('on_oline')[['game_play_id','nfl_id','on_oline']]
    df_tracking = (
        df_tracking.merge(
            on_oline,
            on=['game_play_id','nfl_id'],
            how='left',
            suffixes=('','_new')
        )
    )
    df_tracking['on_oline'] = df_tracking['on_oline_new'].fillna(df_tracking['on_oline'])
    df_tracking.drop(columns='on_oline_new', inplace=True)

    oline_y_min_left = (
        df_tracking
        .query('on_oline and x < center_x_at_ls and frame_id == last_line_set_fid')
        [['game_play_id','y']]
        .groupby('game_play_id')
        .y.min()
        .reset_index()
        .rename(columns={'y':'extra_on_oline_y_min_left_at_ls'})
    )
    oline_y_min_right = (
        df_tracking
        .query('on_oline and x > center_x_at_ls and frame_id == last_line_set_fid')
        [['game_play_id','y']]
        .groupby('game_play_id')
        .y.min()
        .reset_index()
        .rename(columns={'y':'extra_on_oline_y_min_right_at_ls'})
    )
    df_tracking = (
        df_tracking
        .drop(columns=['extra_on_oline_y_min_left_at_ls','extra_on_oline_y_min_right_at_ls'])
        .merge(oline_y_min_left, on='game_play_id', how='left')
        .merge(oline_y_min_right, on='game_play_id', how='left')
    )

    extra_on_oline = extra_on_oline.query('~on_oline')
    del extra_on_online_x_min, extra_on_online_x_max, on_oline, oline_y_min_left, oline_y_min_right

del extra_on_oline

In [330]:
# Add feature for yards behind oline at line set
line_set = (
    df_tracking
    .query('frame_id == last_line_set_fid')
    [['game_play_id','nfl_id','y','x','center_x_at_ls','oline_y_min_left_at_ls','oline_y_min_right_at_ls']]
    .rename(columns={'y':'y_at_line_set'})
)
line_set['yards_behind_oline'] = np.where(
    line_set['x'] < line_set['center_x_at_ls'],
    np.where(
        line_set['y_at_line_set'] < line_set['oline_y_min_left_at_ls'],
        line_set['oline_y_min_left_at_ls'] - line_set['y_at_line_set'],
        0
    ),
    np.where(
        line_set['y_at_line_set'] < line_set['oline_y_min_right_at_ls'],
        line_set['oline_y_min_right_at_ls'] - line_set['y_at_line_set'],
        0
    )
)
df_tracking = df_tracking.merge(
    line_set[['game_play_id','nfl_id','yards_behind_oline']],
    on=['game_play_id','nfl_id'], 
    how='left'
)

In [331]:
# Classify Off-Line Y Alignment
last_line_set = (
    df_tracking
    .query('frame_id == last_line_set_fid and ~position_by_loc.isin(["LT","LG","C","RG","RT"]) and on_oline')
    .query('(y == extra_on_oline_y_min_left_at_ls) or ' +\
           '(y == extra_on_oline_y_min_right_at_ls)')
    .query('yards_behind_oline > .5')
    [['game_play_id','nfl_id']]
)

last_line_set['initial_alignment'] = 'Off-Line Y'

df_tracking = df_tracking.merge(
    last_line_set[['game_play_id','nfl_id','initial_alignment']], 
    on=['game_play_id','nfl_id'], 
    how='left'
)

In [332]:
if 'initial_alignment' in df_motion.columns:
    df_motion.drop(columns=['initial_alignment'], inplace=True)

df_motion = df_motion.merge(
    (
        df_tracking
        .query('motion_player')
        [['game_play_id','nfl_id','initial_alignment']]
        .drop_duplicates(['game_play_id','nfl_id'])
    ),
    on=['game_play_id','nfl_id'],
    how='left'
)

In [333]:
xy_first_motion = (
    df_motion
    .query('motion_frame')
    .sort_values('frame_id')
    .drop_duplicates('game_play_id', keep='first')
    [['game_play_id','x','y']]
    .rename(columns={'x':'x_first_motion','y':'y_first_motion'})
)
xy_last_motion = (
    df_motion
    .query('motion_frame')
    .sort_values('frame_id', ascending=False)
    .drop_duplicates('game_play_id', keep='first')
    [['game_play_id','x','y']]
    .rename(columns={'x':'x_last_motion','y':'y_last_motion'})
)
if 'x_first_motion' in df_motion.columns:
    df_motion.drop(columns=['x_first_motion','y_first_motion','x_last_motion','y_last_motion'], inplace=True)
df_motion = df_motion.merge(xy_first_motion, on='game_play_id', how='left')
df_motion = df_motion.merge(xy_last_motion, on='game_play_id', how='left')

if 'dx' in df_motion.columns:
    df_motion.drop(columns=['dx'], inplace=True)
if 'dy' in df_motion.columns:
    df_motion.drop(columns=['dy'], inplace=True)

df_motion['dx'] = df_motion['x_first_motion'] - df_motion['x_last_motion']
df_motion['dy'] = df_motion['y_first_motion'] - df_motion['y_last_motion']

In [334]:
last_line_set_fid = (
    df_tracking
    .drop_duplicates('game_play_id', keep='last')
    [['game_play_id','last_line_set_fid']]
)
if 'last_line_set_fid' in df_motion.columns:
    df_motion.drop(columns='last_line_set_fid', inplace=True)
df_motion = df_motion.merge(last_line_set_fid, on='game_play_id', how='left')


side_pre_motion = (
    df_motion
    .query('frame_id == last_line_set_fid')
    [['game_play_id','oline_side']]
    .rename(columns={'oline_side':'oline_side_pre_motion'})
)
if 'oline_side_pre_motion' in df_motion.columns:
    df_motion.drop(columns='oline_side_pre_motion', inplace=True)
df_motion = df_motion.merge(side_pre_motion, on='game_play_id', how='left')

In [335]:
in_type_motion_gids = (
    df_motion
    .drop_duplicates('game_play_id')
    .query(
        '(motion_group.isna() or motion_group=="nan") and ' +\
        'initial_alignment == "Off-Line Y" and ' +\
        'ball_snap_before_passing_center and ' +\
        '0.75 <= dx_dy_ratio <= 3 and ' +\
        '1 <= dy <= 3 and ' +\
        '((oline_side_pre_motion == "outside-left" and -3 <= dx <= -1) or ' +\
        ' (oline_side_pre_motion == "outside-right" and 1 <= dx <= 3))'
    )
    .game_play_id
    .unique().tolist()
)

df_motion.loc[df_motion['game_play_id'].isin(in_type_motion_gids), 'motion_group'] = 'In'
df_motion.loc[df_motion['game_play_id'].isin(in_type_motion_gids), 'motion_sub_group'] = 'Off-Line Y'

# FB Shuffle Motion

In [336]:
if 'offense_formation' in df_motion.columns:
    df_motion.drop(columns='offense_formation', inplace=True)
df_motion = df_motion.merge(
    df_play[['game_play_id','offense_formation']],
    on='game_play_id',
    how='left'
)

if 'center_x_at_ls' in df_motion.columns:
    df_motion.drop(columns='center_x_at_ls', inplace=True)
df_motion = df_motion.merge(
    df_tracking[['game_play_id','center_x_at_ls']].drop_duplicates('game_play_id'),
    on='game_play_id',
    how='left'
)

if 'qb_y_at_ls' in df_motion.columns:
    df_motion.drop(columns='qb_y_at_ls', inplace=True)
df_motion = df_motion.merge(
    (
        df_tracking.query('position_by_loc == "QB" and frame_id == last_line_set_fid')
        [['game_play_id', 'y']]
        .rename(columns={'y': 'qb_y_at_ls'})
    ),
    on='game_play_id',
    how='left'
)

if 'rb_y_at_ls' in df_motion.columns:
    df_motion.drop(columns='rb_y_at_ls', inplace=True)
df_motion = df_motion.merge(
    (
        df_tracking.query('position_by_loc == "RB" and frame_id == last_line_set_fid')
        [['game_play_id', 'y']]
        .rename(columns={'y': 'rb_y_at_ls'})
    ),
    on='game_play_id',
    how='left'
)

if 'y_at_ls' in df_motion.columns:
    df_motion.drop(columns='y_at_ls', inplace=True)
df_motion = df_motion.merge(
    (
        df_tracking.query('frame_id == last_line_set_fid')
        [['game_play_id', 'nfl_id', 'y']]
        .rename(columns={'y': 'y_at_ls'})
    ),
    on=['game_play_id', 'nfl_id'],
    how='left'
)

df_motion['motion_starts_n_frames_before_snap'] = df_motion['ball_snap_fid'] - df_motion['first_motion_fid']

df_motion['abs_dx_center_at_ls'] = (df_motion['center_x_at_ls'] - df_motion['x_first_motion']).abs()

In [337]:
fb_shuffle_gides = (
    df_motion
    .drop_duplicates('game_play_id')
    .query(
        'position_by_loc == "FB" and ' +\
        'offense_formation == "I_FORM" and ' +\
        'motion_starts_n_frames_before_snap <= 10 and ' +\
        'dx_dy_ratio >= 10 and ' +\
        'abs_dx_center_at_ls <= .5 and ' +\
        'rb_y_at_ls < y_at_ls < qb_y_at_ls'
    )
    .game_play_id
    .unique().tolist()
)

df_motion.loc[df_motion['game_play_id'].isin(fb_shuffle_gides), 'motion_group'] = 'Shuffle'
df_motion.loc[df_motion['game_play_id'].isin(fb_shuffle_gides), 'motion_sub_group'] = 'FB'

# Value Counts

In [338]:
df_motion.loc[df_motion.motion_group == "nan", 'motion_group'] = np.nan
df_motion.loc[~df_motion.motion_group.isna(), 'motion_sub_group'] = "Standard"

In [339]:
df_motion.query('~motion_group.isna()').drop_duplicates('game_play_id').value_counts(['motion_group','motion_sub_group']).reset_index()

,motion_group,motion_sub_group,0
0,Jet,Standard,84
1,Orbit,Standard,41
2,Fly,Standard,32
3,Glide,Standard,30
4,Over,Standard,30
5,Yo-Yo,Standard,19
6,Shuffle,Standard,8
7,In,Standard,4


In [340]:
# unclassified motion plays 
print(f'n unclassified motion plays: {df_motion.query("motion_group.isna()").game_play_id.nunique()}')
print(f'n motion plays: {df_motion.game_play_id.nunique()}')
print(f'pct unclassified motion plays: {df_motion.query("motion_group.isna()").game_play_id.nunique() / df_motion.game_play_id.nunique() :.2%}')
df_motion.query('motion_group.isna()').game_play_id.unique()

n unclassified motion plays: 17
n motion plays: 265
pct unclassified motion plays: 6.42%


array(['2022090800_2072', '2022090800_529', '2022091101_3315',
       '2022091101_521', '2022091103_4462', '2022091105_1008',
       '2022091105_4352', '2022091105_667', '2022091106_3002',
       '2022091107_3163', '2022091109_2481', '2022091109_3463',
       '2022091109_4075', '2022091109_462', '2022091113_1170',
       '2022091113_1318', '2022091113_3722'], dtype=object)

In [341]:
# gpid = "2022092513_843"
# plot_play_with_speed(
#     df_tracking.query('ball_snap_fid - 30 <= frame_id <= ball_snap_fid + 10 and game_play_id==@gpid'),
#     gpid, 
#     every_other_frame=True, 
#     event_col='event_new',
#     plot_motion=True,
#     highlight_offensive_positions=True,
#     show_motion_frames=True
# )

# Calculate distance traveled in motion pre-snap

In [342]:
def calculate_motion_distance(group):
    """ Calculate the total distance traveled by a player during a motion sequence """
    distances = np.sqrt(np.diff(group['x'])**2 + np.diff(group['y'])**2)
    total_distance = np.sum(distances)
    return total_distance

motion_distances = (
    df_motion
    .query('motion_frame')
    .groupby('game_play_id')
    .apply(calculate_motion_distance)
    .reset_index(name='pre_snap_motion_dist_traveled')
)

motion_distances.head()

if 'pre_snap_motion_dist_traveled' in df_motion.columns:
    df_motion.drop(columns='pre_snap_motion_dist_traveled', inplace=True)

df_motion = df_motion.merge(motion_distances, on='game_play_id', how='left')

# Save Motion Classifications

In [343]:
cols = ['game_play_id', 'nfl_id','position_by_loc','had_rush_attempt','initial_alignment',
        'n_direction_changes', 'same_motion_dir', 'motion_dir_first', 'motion_dir_last', 
        'pre_snap_motion_dist_traveled','motion_group', 'motion_sub_group']
rename = {
    'nfl_id':'motion_nfl_id',
    'position_by_loc':'motion_position',
    'had_rush_attempt':'motion_had_rush_attempt'
}

(
    df_motion
    .drop_duplicates('game_play_id')
    [cols]
    .rename(columns=rename)
    .reset_index(drop=True)
    .to_pickle(join(PROCESSED_DATA_PATH, f'wk{WEEK}', 'motion_plays.pkl'))
)

In [344]:
(    
    df_motion
    .drop_duplicates('game_play_id')
    [cols]
    .rename(columns=rename)
    .reset_index(drop=True)
).shape

(265, 12)